#### Library Imports

In [1]:
# Imports
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import matplotlib_inline.backend_inline as backend_inline
from torchsummary import summary
backend_inline.set_matplotlib_formats("svg")


#### Pytorch device specific configuration ###
# # Pytorch Gpu Configuration for Cuda
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Pytorch Gpu Configuration for directml(AMD GPU)
# import torch_directml

# device = torch_directml.device()

# Set default device
torch.set_default_device(device)

#### Data Import and Data Normalization

In [2]:
# Import Dataset
data = np.loadtxt("../Datasets/mnist_train_small.csv", delimiter=",")

# Extract data and labels
labels = data[:, 0]
data = data[:, 1:]
print("Data shape:", data.shape, "\nLabels shape:", labels.shape)

# Data normalization
dataNorm = data / np.max(data)

# Reshape the data to 2D
dataNorm = dataNorm.reshape(dataNorm.shape[0], 1, 28, 28)

# Print data final shape
print("Data final shape:", dataNorm.shape)

Data shape: (20000, 784) 
Labels shape: (20000,)
Data final shape: (20000, 1, 28, 28)


#### Create Train/Test groups using dataloaders

In [3]:
# Step 1: Convert data into tensors
dataT = torch.tensor(dataNorm).float()
labelsT = torch.tensor(labels).long()

# Step 2: Train and Test split
train_data, test_data, train_labels, test_labels = train_test_split(dataT, labelsT, test_size=0.1)

# Step 3: Convert data into PyTorch Datasets
train_data = TensorDataset(train_data, train_labels)
test_data = TensorDataset(test_data, test_labels)

# Step 4: Create DataLoaders for Train and Test sets
batch_size = 32
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, drop_last=True, generator=torch.Generator(device=device))
test_loader = DataLoader(test_data, batch_size=test_data.tensors[0].shape[0], shuffle=False, drop_last=False, generator=torch.Generator(device=device))

In [4]:
# Check the shape of the data in the DataLoader
train_loader.dataset.tensors[0].shape, test_loader.dataset.tensors[0].shape

(torch.Size([18000, 1, 28, 28]), torch.Size([2000, 1, 28, 28]))

#### Create Deep Learning Model

In [6]:
# Define a function to create a CNN model using class
def create_cnn_model(printToggle=False):
    class MNIST_CNN_Model(nn.Module):
        def __init__(self, printToggle):
            super().__init__()
            self.print = printToggle

            # Convolutional layers
            self.conv1 = nn.Conv2d(1, 10, kernel_size=5, stride=1, padding=1)
            # Size = np.floor((28 - 5 + 2 * 1) / 1) + 1 = 26/2 = 13 (/2 b/c max pool)
            self.conv2 = nn.Conv2d(10, 20, kernel_size=5, stride=1, padding=1)
            # Size = np.floor((13+2*1-5) / 1) = 11/2 = 5 (/2 b/c max pool)

            # Fully connected layers
            expected_size = np.floor((5 + 2 * 0 -1)/1) + 1
            expected_size = 20 * int(expected_size ** 2)

            self.fc1 = nn.Linear(expected_size, 50)

            # Output layer
            self.out = nn.Linear(50, 10)

        # Forward Pass
        def forward(self, x):

            print(f'Input: {x.shape}') if self.print else None
            # Convolution 1 -> Maxpool -> ReLU
            x = F.relu(F.max_pool2d(self.conv1(x), 2))
            print(f'After Conv1: {x.shape}') if self.print else None

            # Convolution 2 -> Maxpool -> ReLU
            x = F.relu(F.max_pool2d(self.conv2(x), 2))
            print(f'After Conv2: {x.shape}') if self.print else None

            # Reshape for Linear layer
            nUnits = x.shape.numel() / x.shape[0]
            x = x.view(-1, int(nUnits))
            print(f'After vectorization: {x.shape}') if self.print else None

            # Liniear layers -> ReLU
            x = F.relu(self.fc1(x))
            print(f'After FC1: {x.shape}') if self.print else None
            # Output layer
            x = self.out(x)
            print(f'Output: {x.shape}') if self.print else None

            return x

    # Create model instance
    net = MNIST_CNN_Model(printToggle=printToggle)

    # Loss Funtion
    lossFun = nn.CrossEntropyLoss()

    # Optimizer
    optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

    return net, lossFun, optimizer